In [1]:
import os
import sys
import vtk
import time
import numpy as np
import pyvista as pv
import pymeshfix as mf
import matplotlib.pyplot as plt

from pyacvd import Clustering
from scipy.spatial import KDTree
from matplotlib.colors import ListedColormap


In [63]:
mc1_path = '/Users/mkuczyns/Downloads/DYNACT2_200_WBCT_CROP_PERI_MC1_BB_REORIENT_ABAD_TRANSF_ROTATED_BB_CROP.vtk'
trp_path = '/Users/mkuczyns/Downloads/DYNACT2_200_WBCT_CROP_PERI_TRP_BB_REORIENT_ABAD_TRANSF_ROTATED.vtk'
trp_quad = '/Users/mkuczyns/Downloads/DYNACT2_200_WBCT_CROP_PERI_TRP_BB_REORIENT_ABAD_TRANSF_Q.vtk'

reader_mc1 = vtk.vtkPolyDataReader()
reader_mc1.SetFileName(mc1_path)
reader_mc1.Update()

reader_trp = vtk.vtkPolyDataReader()
reader_trp.SetFileName(trp_path)
reader_trp.Update()

mc1_mesh = pv.PolyData(reader_mc1.GetOutput())
trp_mesh = pv.PolyData(reader_trp.GetOutput())


clean_mc1 = vtk.vtkCleanPolyData()
clean_mc1.SetInputConnection(reader_mc1.GetOutputPort())
clean_mc1.Update()

clean_trp = vtk.vtkCleanPolyData()
clean_trp.SetInputConnection(reader_trp.GetOutputPort())
clean_trp.Update()

smooth_mc1 = vtk.vtkSmoothPolyDataFilter()
smooth_mc1.SetInputConnection(clean_mc1.GetOutputPort())
smooth_mc1.SetNumberOfIterations(50)
smooth_mc1.SetRelaxationFactor(0.2)
smooth_mc1.FeatureEdgeSmoothingOff()
smooth_mc1.BoundarySmoothingOn()
smooth_mc1.Update()

smooth_trp = vtk.vtkSmoothPolyDataFilter()
smooth_trp.SetInputConnection(clean_trp.GetOutputPort())
smooth_trp.SetNumberOfIterations(50)
smooth_trp.SetRelaxationFactor(0.2)
smooth_trp.FeatureEdgeSmoothingOff()
smooth_trp.BoundarySmoothingOn()
smooth_trp.Update()

smooth_mc1 = pv.wrap(smooth_mc1.GetOutput())
smooth_trp = pv.wrap(smooth_trp.GetOutput())

mc1_mesh = pv.read(mc1_path)
trp_mesh = pv.read(trp_path)
trp_quad_mesh = pv.read(trp_quad)

mc1_mesh = mc1_mesh.clean()
trp_mesh = trp_mesh.clean()
trp_quad_mesh = trp_quad_mesh.clean()


In [56]:
# Resampling meshes to have uniform polygons using voronoi clustering
cluster_mc1 = Clustering(mc1_mesh)
cluster_mc1.subdivide(2)
cluster_mc1.cluster(10000)
remesh_mc1 = cluster_mc1.create_mesh()

cluster_trp = Clustering(trp_mesh)
cluster_trp.subdivide(2)
cluster_trp.cluster(10000)
remesh_trp = cluster_trp.create_mesh()


In [57]:
# Mesh smoothing
smooth_mc1 = mc1_mesh.smooth(n_iter=200)
smooth_trp = trp_mesh.smooth(n_iter=200)

# cluster_lunate = Clustering(smooth_lunate)
# cluster_lunate.subdivide(2)
# cluster_lunate.cluster(2000)
# smooth_lunate = cluster_lunate.create_mesh()

# cluster_scaphoid = Clustering(smooth_scaphoid)
# cluster_scaphoid.subdivide(2)
# cluster_scaphoid.cluster(2000)
# smooth_scaphoid = cluster_scaphoid.create_mesh()

p = pv.Plotter()
p.add_mesh(smooth_mc1, color=True, smooth_shading=True)
p.add_mesh(smooth_trp, color=True, smooth_shading=True)
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [16]:
# USING RAY TRACING
# Compute normals
normals = smooth_trp.compute_normals(point_normals=True, cell_normals=False, auto_orient_normals=True)

normals["Proximity"] = np.empty(smooth_trp.n_points)

for i in range(normals.n_points):
    p = normals.points[i]
    vec = normals["Normals"][i] * normals.length
    p0 = p - vec
    p1 = p + vec
    ip, ic = smooth_mc1.ray_trace(p0, p1, first_point=True)
    dist = np.sqrt(np.sum((ip - p)**2))
#     print(dist)
    normals["Proximity"][i] = dist

# Replace zeros with nans
mask = normals["Proximity"] == 0
normals["Proximity"][mask] = np.nan
# np.nanmean(smoothN["distances"])

# print(normals["Proximity"][mask])
# import sys
# sys.exit()

mesh = normals
trp_contact = mesh.threshold(value=(0, 1.5), scalars="Proximity").extract_surface()
# mesh["PointIDs"] = np.arange(mesh.n_points)
# evaluated_lunate = mesh.threshold(value=(0, 2.0), scalars="Proximity").extract_surface()

# Compute the area and COM of the extracted surface (i.e., the surface we consider as joint contact)
# Area is computed in units of pixel^2 so we convert to mm^2 by mutiplying by spacing
com = trp_contact.center_of_mass(scalars_weight=True)
area = round(trp_contact.area, 2)

print('Area : ' + str(area))
print('Center of mass: ' + str(com))

p = pv.Plotter()
p.add_mesh(trp_contact, scalars="Proximity", smooth_shading=True)
p.add_mesh(smooth_trp, color=True, opacity=0.75, smooth_shading=True)
p.show()

Area : 24.62
Center of mass: [nan nan nan]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [49]:
plotter = pv.Plotter(off_screen=True)
# plotter.add_mesh(smooth_lunate.threshold(value=(0, 2.0), scalars="Proximity").extract_surface(), scalars="Proximity", clim=[0, 2], cmap="gist_rainbow", smooth_shading=True)
plotter.add_mesh(trp_contact, scalars="Proximity", clim=[0, 3], cmap="hsv", smooth_shading=True)
plotter.add_mesh(trp_mesh, show_edges=False, opacity=0.3, smooth_shading=True, color='#F9F6EE')
# plotter.add_mesh(scaphoid_mesh, show_edges=True, opacity=0.15, smooth_shading=True, color='#F9F6EE')
# plotter.add_points(com, render_points_as_spheres=True, point_size=15.0, color='red')
plotter.camera_position = 'xz'
plotter.camera.azimuth = 65
plotter.add_text('Surface Area = ' + str(area) + ' mm^2', position=[0,675], font='times', color='white')
plotter.add_text('COM = ' + str(com), font='times', color='white')
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [72]:
# USING KD-TREE
tree = KDTree(smooth_mc1.points)
d, idx = tree.query(smooth_trp.points)
smooth_trp["Proximity"] = d
# np.mean(d)

trp_contact = smooth_trp.threshold(value=(0, 3.0), scalars="Proximity").extract_surface()
trp_contact.save("/Users/mkuczyns/Downloads/frame_1_contact.vtk")
smooth_trp.save("/Users/mkuczyns/Downloads/frame_1_smooth.vtk")
# trp_contact['Proximity'] = trp_contact['Proximity'] - 1.5

p = pv.Plotter()
# p.add_mesh(smooth_trp, color=True, opacity=0.25, smooth_shading=True)
p.add_mesh(trp_quad_mesh, color="yellow", opacity=0.25, smooth_shading=True)
p.add_mesh(trp_contact, scalars="Proximity", clim=[0, 3], cmap="hsv", smooth_shading=True)
p.show()

com = trp_contact.center_of_mass(scalars_weight=True)
area = round(trp_contact.area, 2)

print('Area : ' + str(area))
print('Center of mass: ' + str(com))

# smooth_lunate["PointIDs"] = np.arange(smooth_lunate.n_points)
# evaluated_lunate = smooth_lunate.threshold(value=(0, 2.0), scalars="Proximity").extract_surface()

# evaluated_lunate['Proximity'] = evaluated_lunate['Proximity'] - 2.0

# com = evaluated_lunate.center_of_mass(scalars_weight=True)
# area = round(evaluated_lunate.area, 2)

# print('Area : ' + str(area*0.25))
# print('Center of mass: ' + str(com))

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Area : 173.89
Center of mass: [  0.38224066   1.53332819 -20.09800057]


In [62]:
# USING CLOSEST POINTS
print(smooth_trp.points)
closest_cells, closest_points = smooth_mc1.find_closest_cell(smooth_trp.points, True)
d_exact = np.linalg.norm(smooth_trp.points - closest_points, axis=1)
smooth_trp["Proximity"] = d_exact

mc1_contact = smooth_trp.threshold(value=(0, 1.5), scalars="Proximity").extract_surface()

p = pv.Plotter()
p.add_mesh(mc1_contact, scalars="Proximity", smooth_shading=True)
p.add_mesh(smooth_trp, color=True, opacity=0.75, smooth_shading=True)
p.show()

com = mc1_contact.center_of_mass(scalars_weight=True)
area = round(mc1_contact.area, 2)

print('Area : ' + str(area))
print('Center of mass: ' + str(com))

# smooth_lunate["PointIDs"] = np.arange(smooth_lunate.n_points)
# evaluated_lunate = smooth_lunate.threshold(value=(0, 2.0), scalars="Proximity").extract_surface()

# evaluated_lunate['Proximity'] = evaluated_lunate['Proximity'] - 2.0

# com = evaluated_lunate.center_of_mass(scalars_weight=True)
# area = round(evaluated_lunate.area, 2)

# print('Area : ' + str(area*0.25))
# print('Center of mass: ' + str(com))

[[-12.611417    -0.7736051  -22.038797  ]
 [-12.655273    -0.59322554 -22.215399  ]
 [-12.60784     -0.5857858  -21.9914    ]
 ...
 [  2.5471044    5.623249   -15.373603  ]
 [  2.6546803    5.8009453  -15.332419  ]
 [  2.78131      6.1918     -15.4273    ]]


TypeError: find_closest_cell() takes 2 positional arguments but 3 were given

In [48]:
plotter = pv.Plotter(off_screen=True)
# plotter.add_mesh(smooth_lunate.threshold(value=(0, 2.0), scalars="Proximity").extract_surface(), scalars="Proximity", clim=[0, 2], cmap="gist_rainbow", smooth_shading=True)
plotter.add_mesh(evaluated_lunate, scalars="Proximity", clim=[0, 2], cmap="gist_rainbow", smooth_shading=True)
plotter.add_mesh(lunate_mesh, show_edges=True, opacity=0.15, smooth_shading=True, color='#F9F6EE')
plotter.add_mesh(scaphoid_mesh, show_edges=True, opacity=0.15, smooth_shading=True, color='#F9F6EE')
# plotter.add_points(com, render_points_as_spheres=True, point_size=15.0, color='red')
plotter.camera_position = 'xz'
plotter.camera.azimuth = 65
plotter.add_text('Surface Area = ' + str(area) + ' mm^2', position=[0,675], font='times', color='white')
plotter.add_text('COM = ' + str(com), font='times', color='white')
plotter.show()


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)